In [1]:
import requests
import re
from bs4 import BeautifulSoup
from urllib.request import Request, urlopen
import json

In [2]:
# soup = BeautifulSoup(html, "lxml")

In [3]:
# soup = BeautifulSoup(html, features="xml")

In [4]:

# # extract links from a page, credit: https://pythonspot.com/extract-links-from-webpage-beautifulsoup/
# # I had to <pip3 install lxml> to get this to work
# def getLinks(url):
#     html_page = urlopen(url)
#     soup = BeautifulSoup(html_page, features="lxml")
#     links = []

#     for link in soup.findAll('a', attrs={'href': re.compile("^/Movie Scripts/")}):
#         links.append(link.get('href'))

#     return links

# # this page is a list of all the movies IMSDb has on file for the Sci-Fi genre
# print( getLinks("https://imsdb.com/genre/Sci-Fi") )

# # Now, we need to wade through list of links generated by getLinks and extract
# # the urls for just the scripts, like is down below:

# # this is just the script of a movie
# URL = "https://imsdb.com/scripts/12-Monkeys.html"
# page = requests.get(URL)

# soup = BeautifulSoup(page.content, "html.parser")
# script = soup.find('pre')

# # print(script.text.strip())

### Uncomment the line below if you change the scraped genre in the scrape_scripts.py file

In [5]:
# !python scrape_scripts.py

In [6]:
#print(script.text.strip())

In [7]:
# blackpanther = "https://imsdb.com/scripts/Black-Panther.html"
# page_bp = requests.get(blackpanther)

# soup_bp = BeautifulSoup(page_bp.content, "html.parser")
# script_bp = soup_bp.find('pre')

In [8]:
# print(script_bp.text.strip())

In [9]:
# !pip install convokit

In [10]:
# from convokit import Corpus, download

In [11]:
# corpus = Corpus(filename=download("movie-corpus"))

In [12]:
from pandas.io.json import json_normalize

In [13]:
path_to_file = "scripts.txt"

In [14]:
# Read, then decode for py2 compat.
text = open(path_to_file, 'rb').read().decode(encoding='utf-8')
# length of text is the number of characters in it
print('Length of text: {} characters'.format(len(text)))

Length of text: 6540867 characters


In [15]:
print(text[:250])

Quentin Tarantino's

                         R E S E R V O I R   D O G S








       October 22, 1990

                               -----------------


       This movie is dedicated to these following sources of

       inspi


In [16]:
# temp = open("movie-corpus/corpus.json")
# datum = json.load(temp)

In [17]:
# corpus = json_normalize(datum)

In [18]:
# corpus

In [19]:
# script_bp = script_bp.text.strip()

In [20]:
import tensorflow as tf
from tensorflow.keras.layers.experimental import preprocessing

import numpy as np
import os
import time

In [21]:
vocab = sorted(set(text.strip()))
print('{} unique characters'.format(len(vocab)))

96 unique characters


In [22]:
example_texts = ['abcdefg', 'xyz']

chars = tf.strings.unicode_split(example_texts, input_encoding='UTF-8')
chars

<tf.RaggedTensor [[b'a', b'b', b'c', b'd', b'e', b'f', b'g'], [b'x', b'y', b'z']]>

In [23]:
ids_from_chars = preprocessing.StringLookup(
    vocabulary=list(vocab))

In [24]:
ids = ids_from_chars(chars)
ids

<tf.RaggedTensor [[62, 63, 64, 65, 66, 67, 68], [85, 86, 87]]>

In [25]:
chars_from_ids = tf.keras.layers.experimental.preprocessing.StringLookup(
    vocabulary=ids_from_chars.get_vocabulary(), invert=True)

In [26]:
chars = chars_from_ids(ids)
chars

<tf.RaggedTensor [[b'a', b'b', b'c', b'd', b'e', b'f', b'g'], [b'x', b'y', b'z']]>

In [27]:
tf.strings.reduce_join(chars, axis=-1).numpy()

array([b'abcdefg', b'xyz'], dtype=object)

In [28]:
def text_from_ids(ids):
    return tf.strings.reduce_join(chars_from_ids(ids), axis=-1)

In [29]:
all_ids = ids_from_chars(tf.strings.unicode_split(text, 'UTF-8'))
all_ids

<tf.Tensor: shape=(6540867,), dtype=int64, numpy=array([49, 82, 66, ..., 53, 50, 37])>

In [30]:
ids_dataset = tf.data.Dataset.from_tensor_slices(all_ids)

In [31]:
for ids in ids_dataset.take(13):
    print(chars_from_ids(ids).numpy().decode('utf-8'))

Q
u
e
n
t
i
n
 
T
a
r
a
n


In [32]:
seq_length = 100
examples_per_epoch = len(text)//(seq_length+1)

In [33]:
examples_per_epoch

64761

In [34]:
sequences = ids_dataset.batch(seq_length+1, drop_remainder=True)

for seq in sequences.take(1):
    print(chars_from_ids(seq))

tf.Tensor(
[b'Q' b'u' b'e' b'n' b't' b'i' b'n' b' ' b'T' b'a' b'r' b'a' b'n' b't'
 b'i' b'n' b'o' b"'" b's' b'\r' b'\n' b'\r' b'\n' b' ' b' ' b' ' b' ' b' '
 b' ' b' ' b' ' b' ' b' ' b' ' b' ' b' ' b' ' b' ' b' ' b' ' b' ' b' '
 b' ' b' ' b' ' b' ' b' ' b' ' b'R' b' ' b'E' b' ' b'S' b' ' b'E' b' '
 b'R' b' ' b'V' b' ' b'O' b' ' b'I' b' ' b'R' b' ' b' ' b' ' b'D' b' '
 b'O' b' ' b'G' b' ' b'S' b'\r' b'\n' b'\r' b'\n' b'\r' b'\n' b'\r' b'\n'
 b'\r' b'\n' b'\r' b'\n' b'\r' b'\n' b'\r' b'\n' b'\r' b'\n' b' ' b' '
 b' ' b' ' b' ' b' ' b' ' b'O'], shape=(101,), dtype=string)


In [35]:
for seq in sequences.take(5):
    print(text_from_ids(seq).numpy())

b"Quentin Tarantino's\r\n\r\n                         R E S E R V O I R   D O G S\r\n\r\n\r\n\r\n\r\n\r\n\r\n\r\n\r\n       O"
b'ctober 22, 1990\r\n\r\n                               -----------------\r\n\r\n\r\n       This movie is dedicat'
b'ed to these following sources of\r\n\r\n       inspiration:\r\n\r\n\r\n                               TIMOTHY C'
b'AREY\r\n\r\n                                ROGER CORMAN\r\n\r\n                                ANDRE DeTOTH\r'
b'\n\r\n                                CHOW YUEN FAT\r\n\r\n                               JEAN LUC GODDARD\r\n'


In [36]:
def split_input_target(sequence):
    input_text = sequence[:-1]
    target_text = sequence[1:]
    return input_text, target_text

In [37]:
split_input_target(list("Tensorflow"))

(['T', 'e', 'n', 's', 'o', 'r', 'f', 'l', 'o'],
 ['e', 'n', 's', 'o', 'r', 'f', 'l', 'o', 'w'])

In [38]:
dataset = sequences.map(split_input_target)

In [39]:
for input_example, target_example in  dataset.take(1):
    print("Input :", text_from_ids(input_example).numpy())
    print("Target:", text_from_ids(target_example).numpy())

Input : b"Quentin Tarantino's\r\n\r\n                         R E S E R V O I R   D O G S\r\n\r\n\r\n\r\n\r\n\r\n\r\n\r\n\r\n       "
Target: b"uentin Tarantino's\r\n\r\n                         R E S E R V O I R   D O G S\r\n\r\n\r\n\r\n\r\n\r\n\r\n\r\n\r\n       O"


In [40]:
# Batch size
BATCH_SIZE = 64

BUFFER_SIZE = 1000

dataset = (
    dataset
    .shuffle(BUFFER_SIZE)
    .batch(BATCH_SIZE, drop_remainder=True)
    .prefetch(tf.data.experimental.AUTOTUNE))

dataset

<PrefetchDataset shapes: ((64, 100), (64, 100)), types: (tf.int64, tf.int64)>

In [41]:
class MyModel(tf.keras.Model):
    def __init__(self, vocab_size, embedding_dim, rnn_units):
        super().__init__(self)
        self.embedding = tf.keras.layers.Embedding(vocab_size, embedding_dim)
        self.gru = tf.keras.layers.GRU(rnn_units,
                                   return_sequences=True, 
                                   return_state=True)
        self.dense = tf.keras.layers.Dense(vocab_size)

    def call(self, inputs, states=None, return_state=False, training=False):
        x = inputs
        x = self.embedding(x, training=training)
        if states is None:
            states = self.gru.get_initial_state(x)
        x, states = self.gru(x, initial_state=states, training=training)
        x = self.dense(x, training=training)

        if return_state:
            return x, states
        else: 
            return x

In [42]:
# Length of the vocabulary in chars
vocab_size = len(vocab)

# The embedding dimension
embedding_dim = 256

# Number of RNN units
rnn_units = 1024

In [43]:
model = MyModel(
    # Be sure the vocabulary size matches the `StringLookup` layers.
    vocab_size=len(ids_from_chars.get_vocabulary()),
    embedding_dim=embedding_dim,
    rnn_units=rnn_units)

In [44]:
for input_example_batch, target_example_batch in dataset.take(1):
    example_batch_predictions = model(input_example_batch)
    print(example_batch_predictions.shape, "# (batch_size, sequence_length, vocab_size)")

(64, 100, 97) # (batch_size, sequence_length, vocab_size)


In [45]:
model.summary()

Model: "my_model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       multiple                  24832     
                                                                 
 gru (GRU)                   multiple                  3938304   
                                                                 
 dense (Dense)               multiple                  99425     
                                                                 
Total params: 4,062,561
Trainable params: 4,062,561
Non-trainable params: 0
_________________________________________________________________


In [46]:
sampled_indices = tf.random.categorical(example_batch_predictions[0], num_samples=1)
sampled_indices = tf.squeeze(sampled_indices,axis=-1).numpy()

In [47]:
sampled_indices

array([11, 23, 86, 68, 82, 44, 92, 46,  4, 42, 24, 82, 23, 56, 18, 12, 34,
       78, 27, 48, 30,  3, 36, 71, 42, 93, 27, 60, 53,  1, 49, 91,  5, 37,
       55, 13, 69,  6, 21, 67, 95, 58, 76,  5, 10, 62, 19, 21, 63, 38, 54,
        9, 85,  5,  1, 65, 23, 33, 85, 32,  5, 11, 11, 41, 47, 69, 25, 17,
       38, 64, 49, 57, 54, 73, 67, 26, 88,  8, 87, 36, 42, 90, 90, 60, 30,
       85, 89,  5, 25, 75, 25, 74, 39, 76, 91, 44, 12, 58, 92, 49])

In [48]:
print("Input:\n", text_from_ids(input_example_batch[0]).numpy())
print()
print("Next Char Predictions:\n", text_from_ids(sampled_indices).numpy())

Input:
 b'.\r\n\r\n                              MR. WHITE\r\n                 You talked to Nice Guy Eddie?  Why\r\n '

Next Char Predictions:
 b'\'5yguL\xc2\x9dN J6u5X0(Bq9P=\rDjJ\xc2\xab9]U\tQ\xc2\x9c!EW)h"3f\xc2\xbdZo!&a13bFV%x!\td5Ax?!\'\'IOh7/FcQYVlf8}$zDJ\xc2\x98\xc2\x98]=x\xc2\x80!7n7mGo\xc2\x9cL(Z\xc2\x9dQ'


In [49]:
loss = tf.losses.SparseCategoricalCrossentropy(from_logits=True)

In [50]:
example_batch_loss = loss(target_example_batch, example_batch_predictions)
mean_loss = example_batch_loss.numpy().mean()
print("Prediction shape: ", example_batch_predictions.shape, " # (batch_size, sequence_length, vocab_size)")
print("Mean loss:        ", mean_loss)

Prediction shape:  (64, 100, 97)  # (batch_size, sequence_length, vocab_size)
Mean loss:         4.5745244


In [51]:
tf.exp(mean_loss).numpy()

96.9819

In [52]:
model.compile(optimizer='adam', loss=loss)

In [53]:
# Directory where the checkpoints will be saved
checkpoint_dir = './deneme2'
# Name of the checkpoint files
checkpoint_prefix = os.path.join(checkpoint_dir, "ckpt_{epoch}")

checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_prefix,
    save_weights_only=True)

In [60]:
EPOCHS = 20
# with 2 epochs, loss is 1.5116

In [61]:
history = model.fit(dataset, epochs=EPOCHS, callbacks=[checkpoint_callback])

Epoch 1/20
1011/1011 [==============================] - 3358s 3s/step - loss: 0.9608
Epoch 2/20
1011/1011 [==============================] - 2818s 3s/step - loss: 0.8225
Epoch 3/20
1011/1011 [==============================] - 2857s 3s/step - loss: 0.7336
Epoch 4/20
1011/1011 [==============================] - 2825s 3s/step - loss: 0.6589
Epoch 5/20
1011/1011 [==============================] - 2790s 3s/step - loss: 0.5995
Epoch 6/20
1011/1011 [==============================] - 2785s 3s/step - loss: 0.5582
Epoch 7/20
1011/1011 [==============================] - 2794s 3s/step - loss: 0.5317
Epoch 8/20
1011/1011 [==============================] - 2773s 3s/step - loss: 0.5153
Epoch 9/20
1011/1011 [==============================] - 2760s 3s/step - loss: 0.5056
Epoch 10/20
1011/1011 [==============================] - 2748s 3s/step - loss: 0.5011
Epoch 11/20
1011/1011 [==============================] - 2752s 3s/step - loss: 0.4980
Epoch 12/20
1011/1011 [==============================] - 2760s 

In [62]:
class OneStep(tf.keras.Model):
    def __init__(self, model, chars_from_ids, ids_from_chars, temperature=1.0):
        super().__init__()
        self.temperature=temperature
        self.model = model
        self.chars_from_ids = chars_from_ids
        self.ids_from_chars = ids_from_chars

    # Create a mask to prevent "" or "[UNK]" from being generated.
        skip_ids = tf.constant([[0], [0]], dtype=tf.int64)
        tmp1 = tf.reshape(skip_ids, shape=(-1,))
        uniques, idx, counts = tf.unique_with_counts(tmp1)
        uniques_ids = tf.expand_dims(uniques, axis=1)
        
        sparse_mask = tf.SparseTensor(
    # Put a -inf at each bad index.
                values=[-float('inf')] * len(uniques_ids),
                indices=uniques_ids,
    # Match the shape to the vocabulary
                dense_shape=[len(ids_from_chars.get_vocabulary())])
        print(sparse_mask)
        
#         #skip_ids = self.ids_from_chars(['','[UNK]'])[:, None]
#         sparse_mask = tf.SparseTensor(
#         # Put a -inf at each bad index.
#             values=[-float('inf')]*len(skip_ids),
#             indices = skip_ids,
#         # Match the shape to the vocabulary
#             dense_shape=[len(ids_from_chars.get_vocabulary())]) 

        self.prediction_mask = tf.sparse.to_dense(sparse_mask)

    @tf.function
    def generate_one_step(self, inputs, states=None):
    # Convert strings to token IDs.
        input_chars = tf.strings.unicode_split(inputs, 'UTF-8')
        input_ids = self.ids_from_chars(input_chars).to_tensor()

    # Run the model.
    # predicted_logits.shape is [batch, char, next_char_logits] 
        predicted_logits, states =  self.model(inputs=input_ids, states=states, 
                                          return_state=True)
    # Only use the last prediction.
        predicted_logits = predicted_logits[:, -1, :]
        predicted_logits = predicted_logits/self.temperature
    # Apply the prediction mask: prevent "" or "[UNK]" from being generated.
        predicted_logits = predicted_logits + self.prediction_mask

    # Sample the output logits to generate token IDs.
        predicted_ids = tf.random.categorical(predicted_logits, num_samples=1)
        predicted_ids = tf.squeeze(predicted_ids, axis=-1)
    
    # Convert from token ids to characters
        predicted_chars = self.chars_from_ids(predicted_ids)

    # Return the characters and model state.
        return predicted_chars, states

In [63]:
one_step_model = OneStep(model, chars_from_ids, ids_from_chars)

SparseTensor(indices=tf.Tensor([[0]], shape=(1, 1), dtype=int64), values=tf.Tensor([-inf], shape=(1,), dtype=float32), dense_shape=tf.Tensor([97], shape=(1,), dtype=int64))


In [58]:
start = time.time()
states = None
next_char = tf.constant(['N\'JOBU:'])
result = [next_char]

for n in range(10000):
    next_char, states = one_step_model.generate_one_step(next_char, states=states)
    result.append(next_char)

result = tf.strings.join(result)
end = time.time()

print(result[0].numpy().decode('utf-8'), '\n\n' + '_'*80)

print(f"\nRun time: {end - start}")

N'JOBU:
Beach another steple oo A donton at a smilf berond.

                         HUNSECKER
                   (smartly)
            Susie, he nob, Sodrice.

HUSSETAR SUSAN

            A supphone from conicesteric the wilited which dead.  Sidney colver.
                   (clauf)
            All that just co.  Hello?

EXT. BRBUTHO, Now you square
CAMERA shokeches her hadge.

ROUM STADD IN  FLOW BROUR HUNSECKER

Sidney darchs off and calms out, the may.  Hut is acancent,
one louseng.  Where is susseal:

                    NORMA
                   D'Ange, J.J. it's over girl we me in
            agaunt!
                   (Walling)
                     (lood.

                     HUNSECKER
                   ((pasting?)
            I last, weskned to retaly.  Sidney poys to do.
             out off,
           Well, is a dirtation, ignowing you
             hears.  What's you got thing.
            expiets to worl with her, Sidney's voice.
                    (franting
in fear, Si

In [59]:
print(type(result))

<class 'tensorflow.python.framework.ops.EagerTensor'>


next steps: find nonsense words, count num nonsense/normal words --> success metric

Code used: [the original script](https://github.com/tarik3333/Movie-script-gen/blob/main/Untitled9.ipynb) and [this answer](https://stackoverflow.com/questions/68939916/tensorflow-text-generation-rnn-example-failing-on-tf-2-6-tf-sparse-to-dense)